# Read All Dataset CSV

In [75]:
import os
import csv
import re
import pandas as pd
import numpy as np

In [139]:
dataset_names=[]
X_trains=[]
y_trains=[]
X_tests=[]

# 收集所有資料夾名稱，按照數字順序排序資料夾名稱
for folder_name in os.listdir("../Competition_data"):
    dataset_names.append(folder_name)
dataset_names = sorted(dataset_names, key=lambda x: int(re.search(r'\d+', x).group()))

for folder_name in dataset_names:
    # print(folder_name)
    X_trains.append(pd.read_csv(f"../Competition_data/{folder_name}/X_train.csv",header=0))
    y_trains.append(pd.read_csv(f"../Competition_data/{folder_name}/y_train.csv",header=0))
    X_tests.append(pd.read_csv(f"../Competition_data/{folder_name}/X_test.csv",header=0))
    
    
    
# print(len(dataset_names))
# print(len(X_trains))  # 49, 代表有 49 個 dataFrame (每個資料集各一個)
# print(len(y_trains))
# print(len(X_tests))
# print(X_trains[0].dtypes)
# print(y_trains[0].dtypes)

In [77]:
# print(X_trains[42])

## Data Preprocessing & Feature Engineering

In [78]:
# from sklearn.preprocessing import OneHotEncoder
# from xgboost import XGBClassifier
# from pyod.models.iforest import IForest
# import pandas as pd
# import numpy as np

# for i in range(len(dataset_names)):
#     # Step 1: 分離數值型和類別型特徵
#     numerical_df = X_trains[i].select_dtypes(include=['float'])  # 數值型特徵
#     categorical_df = X_trains[i].select_dtypes(include=['int'])  # 類別型特徵

#     # Step 2: 使用 PYOD 的 IForest 檢測初始異常值
#     pyod_clf = IForest(contamination=0.1, random_state=42)  # 假設 10% 異常
#     pyod_clf.fit(numerical_df.values)
#     initial_outliers = pyod_clf.predict(numerical_df.values)  # 1 表示異常，0 表示正常

#     # Step 3: 將初始檢測結果作為 XGBoost 的訓練標籤
#     xgb_clf = XGBClassifier(
#         n_estimators=50,
#         random_state=42,

#         eval_metric="logloss",
#     )
#     xgb_clf.fit(numerical_df, initial_outliers)

#     # Step 4: 使用 XGBoost 預測訓練數據中的異常值
#     final_outliers = xgb_clf.predict(numerical_df)
    
#     # Step 5: 過濾異常值
#     is_normal = (final_outliers == 0)  # XGBoost 中 0 表示正常
#     numerical_df = numerical_df[is_normal].reset_index(drop=True)
#     categorical_df = categorical_df[is_normal].reset_index(drop=True)
#     y_trains[i] = y_trains[i].iloc[is_normal].reset_index(drop=True)

#     # Step 6: 對類別型特徵進行 One-Hot 編碼
#     encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
#     categorical_encoded = encoder.fit_transform(categorical_df)
#     categorical_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out())

#     # 合併處理後的數據
#     X_trains[i] = pd.concat([numerical_df, categorical_df], axis=1)

#     # Step 7: 對測試數據進行相同處理
#     numerical_df_test = X_tests[i].select_dtypes(include=['float'])
#     categorical_df_test = X_tests[i].select_dtypes(include=['int'])

#     # 使用 XGBoost 預測測試數據中的異常值
#     test_outliers = xgb_clf.predict(numerical_df_test)
#     is_test_normal = (test_outliers == 0)
#     numerical_df_test = numerical_df_test[is_test_normal].reset_index(drop=True)
#     categorical_df_test = categorical_df_test[is_test_normal].reset_index(drop=True)

#     # 使用已訓練的 One-Hot Encoder 處理類別型特徵
#     categorical_encoded_test = encoder.transform(categorical_df_test)
#     categorical_df_test = pd.DataFrame(categorical_encoded_test, columns=encoder.get_feature_names_out())

#     # 合併測試數據
#     X_tests[i] = pd.concat([numerical_df_test, categorical_df_test], axis=1)

In [79]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer

# # 對每組資料進行處理
# for i in range(len(dataset_names)):
#     # 將連續型資料和數值型資料標準化
#     numerical_df = X_trains[i].select_dtypes(include=['float'])   # 數值型特徵
#     categorical_df = X_trains[i].select_dtypes(include=['int'])   # 類別型特徵（可能為類別特徵）
#     if len(numerical_df.columns) + len(categorical_df.columns) != len(X_trains[i].columns):
#         print('Splitting error')
#     # numerical_df --> normalization
#     scaler = StandardScaler()
#     scaler.fit(numerical_df)
#     numerical_s = scaler.transform(numerical_df)
#     numerical_df = pd.DataFrame(numerical_s, columns=numerical_df.columns)
#     X_trains[i] = pd.concat([numerical_df, categorical_df], axis=1)
    
    
#     numerical_df = X_tests[i].select_dtypes(include=['float'])   # 數值型特徵
#     categorical_df = X_tests[i].select_dtypes(include=['int'])   # 類別型特徵（可能為類別特徵）
#     # 直接照前面用過的 scaler 來分
#     numerical_s = scaler.transform(numerical_df)
#     numerical_df = pd.DataFrame(numerical_s, columns=numerical_df.columns)
#     X_tests[i] = pd.concat([numerical_df, categorical_df], axis=1)

# for i in range(len(dataset_names)):
#     y_trains[i].iloc[:, 0] = pd.to_numeric(y_trains[i].iloc[:, 0])

In [ ]:
# from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
# from sklearn.decomposition import PCA
# from xgboost import XGBClassifier
# import pandas as pd


# importance_selection = 0.8  # 選取重要特徵的比例
# # 遍歷每個數據集進行處理
# for i in range(len(dataset_names)):
#     print(f"Dataset {i} before encoding: {X_trains[i].shape[1]}")
    
#     # 分離數值型和類別型特徵
#     numerical_df = X_trains[i].select_dtypes(include=['float'])  # 數值型特徵
#     categorical_df = X_trains[i].select_dtypes(include=['int'])  # 類別型特徵
    
#     # # One-hot encoding 類別型特徵
#     # encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
#     # categorical_df = pd.DataFrame(encoder.fit_transform(categorical_df), columns=encoder.get_feature_names_out())
    
#     # # 合併處理後的數據
#     # X_trains[i] = pd.concat([numerical_df, categorical_df], axis=1)
    
#     # # 處理測試數據集中的數值型和類別型特徵
#     # numerical_df_test = X_tests[i].select_dtypes(include=['float'])
#     # categorical_df_test = X_tests[i].select_dtypes(include=['int'])
#     # categorical_df_test = pd.DataFrame(encoder.transform(categorical_df_test), columns=encoder.get_feature_names_out())
#     # X_tests[i] = pd.concat([numerical_df_test, categorical_df_test], axis=1)
    
#     # 使用 XGBoost 選取重要特徵
#     xgb_clf = XGBClassifier(
#         n_estimators=100,
#         random_state=42,
#         eval_metric='auc'
#     )
#     xgb_clf.fit(X_trains[i], y_trains[i])
    
#     # 提取特徵重要性
#     importance = xgb_clf.feature_importances_
#     sorted_indices = importance.argsort()[::-1]
#     top_features = sorted_indices[:int(X_trains[i].shape[1] * importance_selection)]
    
#     # 過濾數據，只保留重要特徵
#     X_trains[i] = X_trains[i].iloc[:, top_features]
#     X_tests[i] = X_tests[i].iloc[:, top_features]
#     print(f"Dataset {i} after importance selection: {X_trains[i].shape[1]}")
    
#     # # 數值型特徵標準化
#     # scaler = MinMaxScaler()
#     # X_trains[i] = pd.DataFrame(scaler.fit_transform(X_trains[i]), columns=X_trains[i].columns)
#     # X_tests[i] = pd.DataFrame(scaler.transform(X_tests[i]), columns=X_tests[i].columns)
    
#     # # 使用 PCA 對重要特徵進行降維
#     # pca = PCA(n_components=0.95, random_state=42)  # 保留 95% 累積方差
#     # X_trains[i] = pd.DataFrame(pca.fit_transform(X_trains[i]))
#     # X_tests[i] = pd.DataFrame(pca.transform(X_tests[i]))
#     # print(f"PCA reduced features of Dataset {i} to {X_trains[i].shape[1]}")

# # 確保 y_trains 的標籤轉換為數值格式
# for i in range(len(dataset_names)):
#     y_trains[i].iloc[:, 0] = pd.to_numeric(y_trains[i].iloc[:, 0])


Dataset 0 before encoding: 20
Dataset 0 after importance selection: 16
Dataset 1 before encoding: 24
Dataset 1 after importance selection: 19
Dataset 2 before encoding: 9
Dataset 2 after importance selection: 7
Dataset 3 before encoding: 30
Dataset 3 after importance selection: 24
Dataset 4 before encoding: 19
Dataset 4 after importance selection: 15
Dataset 5 before encoding: 16
Dataset 5 after importance selection: 12
Dataset 6 before encoding: 12
Dataset 6 after importance selection: 9
Dataset 7 before encoding: 12
Dataset 7 after importance selection: 9
Dataset 8 before encoding: 46
Dataset 8 after importance selection: 36
Dataset 9 before encoding: 11
Dataset 9 after importance selection: 8
Dataset 10 before encoding: 62
Dataset 10 after importance selection: 49
Dataset 11 before encoding: 5
Dataset 11 after importance selection: 4
Dataset 12 before encoding: 54
Dataset 12 after importance selection: 43
Dataset 13 before encoding: 57
Dataset 13 after importance selection: 45
Datas

In [591]:
# for i in range(len(dataset_names)):
#     missing_cols = set(X_trains[i].columns) - set(X_tests[i].columns)
#     print(missing_cols)

## train test split & build Model
You can select an appropriate model and perform corresponding hyperparameter tuning.

In [592]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

## MODEL


In [929]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np

class YourTorchModel(nn.Module):
    def __init__(self,input_size):
        super(YourTorchModel, self).__init__()
        # 定義模型層
        self.layer0 = nn.Linear(input_size, 128)
        self.layer1 = nn.Linear(128, 64)
        self.layer2 = nn.Linear(64, 32)
        # self.layer3 = nn.Linear(12, 8)
        self.out = nn.Linear(32, 1)

        self.act_fn = nn.ReLU()
        
        self.bn0 = nn.BatchNorm1d(128)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(32)
        self.bn3 = nn.BatchNorm1d(8)
        self.dropout = nn.Dropout(p=0.5)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):

        x = self.layer0(x)
        x = self.bn0(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.layer1(x)
        x = self.bn1(x)
        x = self.act_fn(x)
        x = self.dropout(x)
        
        x = self.layer2(x)
        x = self.bn2(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        # x = self.layer3(x)
        # x = self.bn3(x)
        # x = self.act_fn(x)
        # x = self.dropout(x)
        x = self.out(x)
        return torch.sigmoid(x)  # Apply sigmoid activation here




In [594]:
#set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Neuron network

In [ ]:
# models = []
# avg_auc = 0
# # 根據不同數據續集訓練模型
# for i in range(len(dataset_names)):
#     # 使用 stratify 將數據分為訓練集和測試集
#     tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
#         X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
#     )
#     input_size = tmp_X_train.shape[1]
#     # 將數據轉換為 PyTorch 張量
#     # 將 DataFrame 轉為 numpy 陣列再轉為 PyTorch 張量
#     # 將數據轉換為 PyTorch 張量並移動到 CUDA (GPU)
#     tmp_X_train = torch.tensor(tmp_X_train.values, dtype=torch.float32).to(device)
#     tmp_y_train = torch.tensor(tmp_y_train.values, dtype=torch.float32).to(device)
#     tmp_X_test = torch.tensor(tmp_X_test.values, dtype=torch.float32).to(device)
#     tmp_y_test = torch.tensor(tmp_y_test.values, dtype=torch.float32).to(device)


#     # 初始化模型、損失函數和優化器
#     # model = YourTorchModel(input_size)      #if you don't have cuda gpu
#     model = YourTorchModel(input_size).to(device)
#     criterion = nn.BCELoss()  # 二元交叉熵損失
    
#     l2_lambda = 0.1  # L2 regularization factor
#     l1_lambda = 0.01  # L1 regularization factor   
#     optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=l2_lambda)  

#     # 訓練迴圈
#     num_epochs = 300
    
#     train_acc = 0
    
#     for epoch in range(num_epochs):
#         model.train()
#         optimizer.zero_grad()
#         outputs = model(tmp_X_train)
#         loss = criterion(outputs, tmp_y_train)

#         # Apply L1 regularization
#         l1_penalty = sum(param.abs().sum() for param in model.parameters())
#         loss = loss + l1_lambda * l1_penalty

#         loss.backward()
#         optimizer.step()
#         #train acc
#         preds = (outputs > 0.5).float()  # 將概率轉換為類別標籤
#         train_acc += (preds == tmp_y_train).float().mean()  # 計算準確率

#     # 評估 - 預測測試集的概率
#     model.eval()
#     with torch.no_grad():
#         tmp_y_prob = model(tmp_X_test).squeeze().cpu().numpy() # 概率預測值
    
#     # 計算 AUC
#     # 經過資料處理後，可能會出現只有一個類別的情況，此時 AUC 會報錯，因此這裡做了一個判斷
#     unique_classes = np.unique(tmp_y_test.cpu().numpy())
#     if len(unique_classes) < 2:
#         print(f"Skipping AUC calculation for this dataset due to single class in y_test.")
#         auc = 0.7  # 或選擇其他合適的值，例如預設值
#     else:
#         auc = roc_auc_score(tmp_y_test.cpu().numpy(), tmp_y_prob)

#     # print(f"{dataset_names[i]} AUC: {auc}")
#     avg_auc += auc
#     models.append(model)

#     train_acc /= num_epochs
#     # 在每個 dataset 結束時打印損失和準確率,and test auc
#     print(f"Dataset {dataset_names[i]}: Train Acc: {train_acc.item():.8f}, Test AUC: {auc:.8f}")   

# print("平均 AUC:", avg_auc / len(dataset_names))

Dataset Dataset_1: Train Acc: 0.66970849, Test AUC: 0.80280830
Dataset Dataset_2: Train Acc: 0.92984736, Test AUC: 0.99366472
Dataset Dataset_3: Train Acc: 0.72200012, Test AUC: 0.71428571
Dataset Dataset_4: Train Acc: 0.92725891, Test AUC: 0.64089347
Dataset Dataset_5: Train Acc: 0.86858177, Test AUC: 0.81147541
Dataset Dataset_6: Train Acc: 0.88998628, Test AUC: 1.00000000
Dataset Dataset_7: Train Acc: 0.91897207, Test AUC: 0.87656250
Dataset Dataset_8: Train Acc: 0.75533777, Test AUC: 0.81597222
Dataset Dataset_9: Train Acc: 0.79602915, Test AUC: 0.89523810
Dataset Dataset_10: Train Acc: 0.62875777, Test AUC: 0.64801865
Dataset Dataset_11: Train Acc: 0.86253685, Test AUC: 0.75000000
Dataset Dataset_12: Train Acc: 0.72742558, Test AUC: 0.64646465
Dataset Dataset_13: Train Acc: 0.87562472, Test AUC: 0.86013986
Dataset Dataset_14: Train Acc: 0.91094935, Test AUC: 1.00000000
Dataset Dataset_15: Train Acc: 0.72751677, Test AUC: 0.66833333
Dataset Dataset_16: Train Acc: 0.93708795, Test A

### 把所有資料訓練集合後訓練(效果不是很好)

In [596]:
# models = []
# avg_auc = 0
# num_epochs = 100  # 訓練迴圈放到最外層
# # 把所有數據集放在一起訓練
# for epoch in range(num_epochs):
#     epoch_train_acc = 0
#     epoch_avg_auc = 0
#     print(f"Epoch {epoch + 1}/{num_epochs}")

#     for i in range(len(dataset_names)):
#         # 使用 stratify 將數據分為訓練集和測試集
#         tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
#             X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
#         )
#         input_size = tmp_X_train.shape[1]

#         # 將數據轉換為 PyTorch 張量並移動到 CUDA (GPU)
#         tmp_X_train = torch.tensor(tmp_X_train.values, dtype=torch.float32).to(device)
#         tmp_y_train = torch.tensor(tmp_y_train.values, dtype=torch.float32).to(device)
#         tmp_X_test = torch.tensor(tmp_X_test.values, dtype=torch.float32).to(device)
#         tmp_y_test = torch.tensor(tmp_y_test.values, dtype=torch.float32).to(device)

#         # 初始化模型、損失函數和優化器
#         # if epoch == 0 and i == 0:
#         model = YourTorchModel(input_size).to(device)
#         criterion = nn.BCELoss()  # 二元交叉熵損失
#         l2_lambda = 0.1
#         optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=l2_lambda)

#         # 訓練
#         model.train()
#         optimizer.zero_grad()
#         outputs = model(tmp_X_train)
#         loss = criterion(outputs, tmp_y_train)
#         loss.backward()
#         optimizer.step()

#         # 計算訓練準確率
#         preds = (outputs > 0.5).float()
#         train_acc = (preds == tmp_y_train).float().mean()
#         epoch_train_acc += train_acc.item() / len(dataset_names)

#         # 評估 - 預測測試集的概率
#         model.eval()
#         with torch.no_grad():
#             tmp_y_prob = model(tmp_X_test).squeeze().cpu().numpy()

#         # 計算 AUC
#         auc = roc_auc_score(tmp_y_test.cpu().numpy(), tmp_y_prob)
#         epoch_avg_auc += auc / len(dataset_names)

#         # 保存模型（若在第一個 epoch 時，保存到模型列表中）
#         if epoch == 0:
#             models.append(model)

#     print(f"Epoch {epoch + 1}: Avg Train Acc: {epoch_train_acc:.8f}, Avg Test AUC: {epoch_avg_auc:.8f}")

# print("整體平均 AUC:", epoch_avg_auc)


In [597]:
# import xgboost as xgb
# from sklearn.datasets import load_breast_cancer
# from sklearn.model_selection import train_test_split
# import pandas as pd
# for i in range(len(dataset_names)):
#     # 這邊做一下 stratify
#     X_train, X_test, y_train, y_test = train_test_split(
#         X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
#     )


#     # 構造 DMatrix
#     dtrain = xgb.DMatrix(X_train, label=y_train)

#     # 固定參數
#     base_params = {
#         'objective': 'binary:logistic',
#         'eta': 0.1,  # Learning rate
#         'eval_metric': 'auc',
#         'seed': 42
#     }

#     # 調參範圍
#     max_depth_values = [3, 4, 5, 6, 7]
#     min_child_weight_values = [1, 2, 3, 4, 5]

#     # 保存結果
#     cv_results = []

#     # 網格搜索
#     for max_depth in max_depth_values:
#         for min_child_weight in min_child_weight_values:
#             # 更新參數
#             params = base_params.copy()
#             params['max_depth'] = max_depth
#             params['min_child_weight'] = min_child_weight

#             # 交叉驗證
#             result = xgb.cv(
#                 params=params,
#                 dtrain=dtrain,
#                 num_boost_round=100,
#                 nfold=5,
#                 early_stopping_rounds=10,
#                 seed=42,
#                 as_pandas=True
#             )

#             # 提取最佳AUC及對應的迭代次數
#             best_auc = result['test-auc-mean'].max()
#             best_iteration = result['test-auc-mean'].idxmax() + 1
#             cv_results.append({
#                 'max_depth': max_depth,
#                 'min_child_weight': min_child_weight,
#                 'best_auc': best_auc,
#                 'best_iteration': best_iteration
#             })

#     # 將結果轉為 DataFrame
#     cv_results_df = pd.DataFrame(cv_results)

#     # 查看結果
#     # print(cv_results_df)

#     # 找到最佳參數組合
#     best_result = cv_results_df.loc[cv_results_df['best_auc'].idxmax()]
#     print(f"最佳參數組合: max_depth={best_result['max_depth']}, min_child_weight={best_result['min_child_weight']}")
#     print(f"AUC: {best_result['best_auc']}, 最佳迭代次數: {best_result['best_iteration']}")


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from shaphypetune import BoostSearch
import numpy as np

# 超参数网格
param_grid = {
    # 'max_depth': [3, 6, 9],
    # 'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],

    # 'gamma': [0, 0.5, 1, 1.5, 2],
    'subsample': [0.6,  0.7,0.8,1.0],
    'colsample_bytree': [0.6, 0.65, 0.7,0.75,0.8,0.9],
    'alpha': [0.01,0.1, 0.5, 1],
    'reg_lambda': [0.01,0.1, 0.5, 1],
}

avg_auc = 0  # 累计 AUC
n_datasets = len(dataset_names)

for i in range(n_datasets):
    print(f"Processing Dataset {i}")
    
    # 划分数据集
    tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
        X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
    )
    
    # 初始化模型
    xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    
    # 使用 BoostSearch 进行超参数优化
    boost_search = BoostSearch(
        estimator=xgb_model,
        param_grid=param_grid,
        n_iter=10,
        greater_is_better=True,
        n_jobs=-1,
        verbose=1
    )
    
    # 寻找最佳模型
    best_model = boost_search.fit(
        tmp_X_train, tmp_y_train.squeeze(),
        eval_set=[(tmp_X_test, tmp_y_test)],  # 添加评估集
        verbose=False
    )
    print(f"Best hyperparameters for Dataset {i}: {best_model.best_params_}")
    
    # 获取预测概率
    tmp_y_prob = best_model.predict_proba(tmp_X_test)[:, 1]
    unique_classes = np.unique(tmp_y_test)
    
    # 检查类别数量
    if len(unique_classes) < 2:
        print(f"Skipping AUC calculation for Dataset {i} due to single class in y_test.")
        auc = 0.7
    else:
        auc = roc_auc_score(tmp_y_test, tmp_y_prob)
    
    avg_auc += auc
    print(f"Dataset {i} AUC: {auc}")

# 计算平均 AUC
print("平均 AUC:", avg_auc / n_datasets)


Processing Dataset 0

1152 trials detected for ('n_estimators', 'subsample', 'colsample_bytree', 'alpha', 'reg_lambda')



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   25.7s
[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed:   38.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Best hyperparameters for Dataset 0: {'n_estimators': 300, 'subsample': 0.8, 'colsample_bytree': 0.6, 'alpha': 0.01, 'reg_lambda': 0.01}
Dataset 0 AUC: 0.7484737484737485
Processing Dataset 1

1152 trials detected for ('n_estimators', 'subsample', 'colsample_bytree', 'alpha', 'reg_lambda')



[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed:   21.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Best hyperparameters for Dataset 1: {'n_estimators': 200, 'subsample': 1.0, 'colsample_bytree': 0.9, 'alpha': 0.1, 'reg_lambda': 0.01}
Dataset 1 AUC: 0.9814814814814814
Processing Dataset 2

1152 trials detected for ('n_estimators', 'subsample', 'colsample_bytree', 'alpha', 'reg_lambda')



...

[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed:   24.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Best hyperparameters for Dataset 47: {'n_estimators': 300, 'subsample': 0.6, 'colsample_bytree': 0.8, 'alpha': 0.01, 'reg_lambda': 0.01}
Dataset 47 AUC: 0.8537037037037037
Processing Dataset 48

1152 trials detected for ('n_estimators', 'subsample', 'colsample_bytree', 'alpha', 'reg_lambda')



[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   16.7s


Best hyperparameters for Dataset 48: {'n_estimators': 300, 'subsample': 1.0, 'colsample_bytree': 0.9, 'alpha': 0.01, 'reg_lambda': 0.01}
Dataset 48 AUC: 0.9383651944627555
平均 AUC: 0.8179353948702601


[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed:   25.7s finished


## Cross vlidation(我的電腦的話一次最多跑3個參數about 5 min)
### 預備JSON存CV最好參數

In [30]:
import json

# 加載現有的參數文件
try:
    with open('best_params.json', 'r') as file:
        best_params = json.load(file)
except FileNotFoundError:
    # 如果文件不存在，初始化一個空字典
    best_params = {}

for i in range(len(dataset_names)):
    best_params.setdefault(f"dataset_{i+1}", {"auc": 0, "params": {}})
    # 保存更新後的 JSON 文件
with open('best_params.json', 'w') as file:
    json.dump(best_params, file, indent=4)

In [837]:

# JSON 文件路徑
json_file_path = 'best_params.json'

# 要刪除的鍵
keys_to_delete = ["max_depth", "min_child_weight"]

# 讀取現有 JSON 文件
try:
    with open(json_file_path, 'r') as file:
        best_params = json.load(file)
except FileNotFoundError:
    print(f"File '{json_file_path}' not found.")
    best_params = {}

# 遍歷每個 dataset 並刪除指定鍵
for dataset, params in best_params.items():
    for key in keys_to_delete:
        if key in params:
            del params[key]
            print(f"Deleted key '{key}' from {dataset}")

# 保存更新後的 JSON 文件
with open(json_file_path, 'w') as file:
    json.dump(best_params, file, indent=4)

print("Updated JSON file saved.")


Updated JSON file saved.


## 先調所有人，把auc差的抓去底下加強訓練

綠色代表這次調整有進步，紅色代表<bad auc

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
# Define your parameter grid for XGBoost or any model you're tuning
param_grid = {
    
    # 'learning_rate': [0.01,0.03,0.05,0.08,0.1,0.5],
    'n_estimators': [100,200,300,400,500],
    # 'gamma': [0, 0.5, 1, 1.5, 2],
    # 'max_depth': [5,6, 7,9],
    # 'max_delta_step': [0,1,2,3,4,5],
    # 'subsample': [0.6, 0.65, 0.7,0.75,0.8,0.9],
    # 'colsample_bytree': [0.6, 0.65, 0.7,0.75,0.8,0.9],
    # 'alpha': [0.01,0.05,0.1, 0.5, 1,1.5],
    # 'reg_lambda': [0.01,0.05,0.1, 0.5, 1,1.5],

    # "min_child_weight": [1,2, 3, 5,7],
}

avg_auc = 0
models = []
retrain_index = []
smote = SMOTE(random_state=42)
# Define a threshold for bad AUC
bad_threshold = 0.8

for i in range(len(dataset_names)):
    tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
        X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
    )
    tmp_X_train,tmp_y_train = smote.fit_resample(tmp_X_train, tmp_y_train)
    # Cross-validation with parameter tuning
    model = XGBClassifier(
        # n_estimators=500, 
        # gamma=1,
        # eta=0.01,
        # max_depth=6, 
        # subsample=0.7, 
        # colsample_bytree=0.6,
        objective='binary:logistic',
        eval_metric='logloss',
        **best_params.get(f"dataset_{i+1}", {}).get("params", {})    # Load the best parameters from the JSON file
        # reg_lambda=1,     # L2 regularization term on weights
        # alpha=0.1,         # L1 regularization term on weights
        
    )
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
    grid_search.fit(tmp_X_train, tmp_y_train.squeeze())
    
    # Get the best model from cross-validation
    best_model = grid_search.best_estimator_
    models.append(best_model)
    print(f"Best parameters for dataset {i+1}: {grid_search.best_params_}")
    # Evaluate the best model
    tmp_y_prob = best_model.predict_proba(tmp_X_test)[:, 1]
    unique_classes = np.unique(tmp_y_test)

    if len(unique_classes) < 2:
        print(f"Skipping AUC calculation for this dataset due to single class in y_test.")
        auc = 0.7
    else:
        auc = roc_auc_score(tmp_y_test, tmp_y_prob)
    avg_auc += auc

    #將準確低於一定值的模型加入list 等等重新
    if auc < bad_threshold:
        retrain_index.append(i)
        print(f"\033[31mAUC for dataset {i+1} with tuned parameters: {auc}\033[0m") 
    
    # 更新或追加參數(if auc better)
    if auc > best_params.get(f"dataset_{i+1}", {}).get('auc', 0):
        print(f"\033[32mAUC for dataset {i+1} with tuned parameters: {auc}\033[0m")     
        best_params[f"dataset_{i+1}"]['auc'] = auc

        if f"dataset_{i+1}" in best_params:
            best_params[f"dataset_{i+1}"]["params"].update(grid_search.best_params_)
        else:
            best_params[f"dataset_{i+1}"]["params"]  = grid_search.best_params_
    # NO UPDATE
    else:
        print(f"AUC for dataset {i+1} with tuned parameters: {auc}")
    # Save the best parameters to a JSON file
    
    with open('best_params.json', 'w') as file:
        json.dump(best_params, file, indent=4)

print(f"Average AUC across all datasets: {avg_auc / len(dataset_names)}")

print("models need to retrain:",retrain_index)



Best parameters for dataset 1: {'n_estimators': 100}
AUC for dataset 1 with tuned parameters: 0.7228327228327229
AUC for dataset 1 with tuned parameters: 0.7228327228327229
Best parameters for dataset 2: {'n_estimators': 100}
AUC for dataset 2 with tuned parameters: 0.9790448343079922
Best parameters for dataset 3: {'n_estimators': 400}
AUC for dataset 3 with tuned parameters: 0.6326530612244898
AUC for dataset 3 with tuned parameters: 0.6326530612244898
Best parameters for dataset 4: {'n_estimators': 100}
AUC for dataset 4 with tuned parameters: 0.7972508591065293
AUC for dataset 4 with tuned parameters: 0.7972508591065293
Best parameters for dataset 5: {'n_estimators': 100}
AUC for dataset 5 with tuned parameters: 0.8934426229508197
Best parameters for dataset 6: {'n_estimators': 100}
AUC for dataset 6 with tuned parameters: 1.0
Best parameters for dataset 7: {'n_estimators': 100}
AUC for dataset 7 with tuned parameters: 0.9953125
Best parameters for dataset 8: {'n_estimators': 100}


#### 把準確率低的抓出來重新訓練

In [820]:
#delete JSON file that auc is lower than 0.75

# JSON 文件路徑
json_file_path = 'best_params.json'

# 要刪除的鍵
keys_to_delete = ["max_depth", "min_child_weight"]

# 讀取現有 JSON 文件
try:
    with open(json_file_path, 'r') as file:
        best_params = json.load(file)
except FileNotFoundError:
    print(f"File '{json_file_path}' not found.")
    best_params = {}

# 遍歷每個 dataset 並刪除指定鍵
for dataset, params in best_params.items():
    if(int(dataset.split("_")[1])-1 in retrain_index):
        for key in keys_to_delete:
            if key in params:
                del params[key]
                print(f"Deleted key '{key}' from {dataset}")

# 保存更新後的 JSON 文件
with open(json_file_path, 'w') as file:
    json.dump(best_params, file, indent=4)

print("Updated JSON file for retrain model saved.")

Deleted key 'min_child_weight' from dataset_3
Deleted key 'min_child_weight' from dataset_10
Deleted key 'min_child_weight' from dataset_11
Deleted key 'min_child_weight' from dataset_15
Deleted key 'min_child_weight' from dataset_24
Deleted key 'min_child_weight' from dataset_30
Deleted key 'min_child_weight' from dataset_38
Updated JSON file for retrain model saved.


In [ ]:
param_grid = {
    
    # 'learning_rate': [0.005, 0.01,0.05,0.1,0.5],
    'n_estimators': [100,200,300,400,500],         #500 400 is mostly good 
    # 'max_depth': [ 2,3,5,6, 7,9,10,12],
    # 'subsample': [0.5, 0.6, 0.7, 0.8, 0.9],
    # 'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
    # 'alpha': [0.01,0.1, 0.5,1.5,2.5,3],
    'reg_lambda': [0.01,0.1,0,3, 0.5,2,2.5,3],
    'gamma': [0,0.3, 0.5,0.7, 1, 1.5, 2,2.5,3],

    # "min_child_weight": [1,2, 3,4, 5],    
}
print(retrain_index)

for i in retrain_index:
    tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
        X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
    )
    tmp_X_train,tmp_y_train = smote.fit_resample(tmp_X_train, tmp_y_train)
    model = XGBClassifier(
    # n_estimators=500, 
    # gamma=1,
    # eta=0.01,
    # max_depth=6, 
    # subsample=0.7, 
    # colsample_bytree=0.6,
    objective='binary:logistic',
    eval_metric='logloss',
    **best_params.get(f"dataset_{i+1}", {}).get("params", {})  # Load the best parameters from the JSON file
    # reg_lambda=1,     # L2 regularization term on weights
    # alpha=0.1,         # L1 regularization term on weights
        
    )
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
    grid_search.fit(tmp_X_train, tmp_y_train.squeeze())
    # model = RandomForestClassifier(
    #     n_estimators=700, n_jobs=-1, random_state=2, max_depth=6,  
    # )
    # Get the best model from cross-validation
    best_model = grid_search.best_estimator_
    
    print(f"Best parameters for dataset {i+1}: {grid_search.best_params_}")

    # Evaluate the best model
    tmp_y_prob = best_model.predict_proba(tmp_X_test)[:, 1]
    # unique_classes = np.unique(tmp_y_test)
    auc = roc_auc_score(tmp_y_test, tmp_y_prob)
    
    # Save the best parameters to JSON
    # 更新或追加參數(if auc better)
    if auc > best_params.get(f"dataset_{i+1}", {}).get('auc', 0):
        print(f"\033[32mAUC for dataset {i+1} with tuned parameters: {auc}\033[0m")
        best_params[f"dataset_{i+1}"]['auc'] = auc

        if f"dataset_{i+1}" in best_params:
            best_params[f"dataset_{i+1}"]["params"].update(grid_search.best_params_)
        else:
            best_params[f"dataset_{i+1}"]["params"]  = grid_search.best_params_
        #update model
        models[i] = best_model
    else:
        print(f"AUC for retrain dataset {i+1} with tuned parameters: {auc}")
    # Save the best parameters to a JSON file
    
    with open('best_params.json', 'w') as file:
        json.dump(best_params, file, indent=4)


[0, 2, 3, 7, 9, 10, 14, 23, 29, 30, 35, 37, 46, 47]
AUC for dataset 1 with tuned parameters: 0.7448107448107448
AUC for retrain dataset 3 with tuned parameters: 0.6938775510204082
AUC for retrain dataset 4 with tuned parameters: 0.6443298969072165
AUC for retrain dataset 8 with tuned parameters: 0.7395833333333334
AUC for retrain dataset 10 with tuned parameters: 0.6853146853146854
AUC for retrain dataset 11 with tuned parameters: 0.5
AUC for retrain dataset 15 with tuned parameters: 0.5916666666666667
AUC for retrain dataset 24 with tuned parameters: 0.706890756302521
AUC for retrain dataset 30 with tuned parameters: 0.6830985915492958
AUC for dataset 31 with tuned parameters: 0.7333333333333335
AUC for retrain dataset 36 with tuned parameters: 0.5946969696969697
AUC for retrain dataset 38 with tuned parameters: 0.6359999999999999
AUC for retrain dataset 47 with tuned parameters: 0.3888888888888889
AUC for retrain dataset 48 with tuned parameters: 0.6888888888888889


In [ ]:
##最後重新train model by best params

models=[]
avg_auc = 0
avg_train = 0

for i in range(len(dataset_names)):
    # 這邊做一下 stratify
    tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
        X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
    )
    tmp_X_train,tmp_y_train = smote.fit_resample(tmp_X_train, tmp_y_train)
    # model = RandomForestClassifier(
    #     n_estimators=700, n_jobs=-1, random_state=2, max_depth=max_deep,  
    # )
    # XGBoost (好像比較容易過擬合，適合大資料集) 
    # print(f"dataset {i+1} params: {best_params.get(f'dataset_{i+1}', {}).get('params', {})}")
    model = XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        
        **best_params.get(f"dataset_{i+1}", {}).get("params", {})  # Load the best parameters from the JSON file
    )
    model.fit(tmp_X_train, tmp_y_train.squeeze())
    
    unique_classes = np.unique(tmp_y_test)
    if len(unique_classes) < 2:
        print(f"Skipping AUC calculation for this dataset due to single class in y_test.")
        auc = 0.7  # 或選擇其他合適的值，例如預設值
    else:
        tmp_y_prob = model.predict_proba(tmp_X_test)[:, 1]
        auc = roc_auc_score(tmp_y_test, tmp_y_prob)
    print(f'auc of dataset {i:2}: \t{auc}')
    avg_auc += auc
    models.append(model)
        
print(f"avg auc :   {avg_auc / len(dataset_names)}")
# print(f"avg auc of maxdeep of {max_deep}:   {avg_auc / len(dataset_names)}")


auc of dataset  0: 	0.8449328449328449
auc of dataset  1: 	1.0
auc of dataset  2: 	0.8367346938775511
auc of dataset  3: 	0.7010309278350515
auc of dataset  4: 	0.9836065573770492
auc of dataset  5: 	0.9926315789473684
auc of dataset  6: 	0.978125


## XGBoost

In [129]:
from imblearn.over_sampling import SMOTE
# 這邊用迴圈跑所有的資料集，並且將每個資料集的資料分成訓練集和測試集
# 並且用 Random Forest 來做分類
# 得到每個資料集的 AUC
# for max_deep in max_deeps:
max_deep  = 5
models=[]
avg_auc = 0
avg_train = 0
smote = SMOTE(random_state=42)
for i in range(len(dataset_names)):
    # 這邊做一下 stratify
    tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
        X_trains[i], y_trains[i], test_size=0.2, random_state=42, 
    )
    tmp_X_train,tmp_y_train = smote.fit_resample(tmp_X_train, tmp_y_train)
    # model = RandomForestClassifier(
    #     n_estimators=700, n_jobs=-1, random_state=2, max_depth=max_deep,  
    # )
    # XGBoost (好像比較容易過擬合，適合大資料集) 
   
    model = XGBClassifier(
        n_estimators=700, 
        # early_stopping_rounds=100,
        eta=0.01,
        
        gamma=1,    # 1 is best
        # min_child_weight = 2,
        scale_pos_weight=2,
        max_depth=10, 
        subsample=0.7, 
        colsample_bytree=0.6,
        objective='binary:logistic',
        eval_metric='logloss',
        reg_lambda=1,     # L2 regularization term on weights
        alpha=0.1         # L1 regularization term on weights
    )
    model.fit(tmp_X_train, tmp_y_train.squeeze())
    
    unique_classes = np.unique(tmp_y_test)
    if len(unique_classes) < 2:
        print(f"Skipping AUC calculation for this dataset due to single class in y_test.")
        auc = 0.7  # 或選擇其他合適的值，例如預設值
    else:
        tmp_y_prob = model.predict_proba(tmp_X_test)[:, 1]
        auc = roc_auc_score(tmp_y_test, tmp_y_prob)
    print(f'auc of dataset {i:2}: \t{auc}')
    avg_auc += auc
    models.append(model)
        
print(f"avg auc :   {avg_auc / len(dataset_names)}")
# print(f"avg auc of maxdeep of {max_deep}:   {avg_auc / len(dataset_names)}")



auc of dataset  0: 	0.8331071913161465
auc of dataset  1: 	0.9995215311004784
auc of dataset  2: 	0.8333333333333334
auc of dataset  3: 	0.4408163265306122
auc of dataset  4: 	0.9100000000000001
auc of dataset  5: 	0.9612244897959183
auc of dataset  6: 	0.9808102345415778
auc of dataset  7: 	0.9381818181818182
auc of dataset  8: 	0.8942307692307692
auc of dataset  9: 	0.8102564102564103
auc of dataset 10: 	0.28571428571428575
auc of dataset 11: 	0.9904761904761904
auc of dataset 12: 	0.95
auc of dataset 13: 	1.0
auc of dataset 14: 	0.7525000000000001
auc of dataset 15: 	0.9995215311004784
auc of dataset 16: 	0.9259259259259259
auc of dataset 17: 	1.0
auc of dataset 18: 	0.9929453262786596
auc of dataset 19: 	0.9333333333333333
auc of dataset 20: 	0.9953379953379954
auc of dataset 21: 	0.8562091503267973
auc of dataset 22: 	0.9512411347517731
auc of dataset 23: 	0.5872093023255814
auc of dataset 24: 	1.0
auc of dataset 25: 	0.8499999999999999
auc of dataset 26: 	1.0
auc of dataset 27: 	

## Inference Model

In [135]:
# #給neural network 用
# y_predicts = []
# for i in range(len(dataset_names)):
#     # Get the test data and convert to tensor if needed
#     X_test = torch.tensor(X_tests[i].values, dtype=torch.float32).to(device)  # Move to the same device as the model

#     # Put model in evaluation mode
#     models[i].eval()
    
#     # Disable gradient calculation for inference
#     with torch.no_grad():
#         # Pass data through the model and apply sigmoid to get probabilities
#         logits = models[i](X_test)  # logits will be on the same device as the model
#         y_predict_proba = torch.sigmoid(logits).cpu().numpy().flatten()  # Convert to numpy array

#     # Store the predictions as a DataFrame
#     df = pd.DataFrame(y_predict_proba, columns=['y_predict_proba'])
#     y_predicts.append(df)
#     #print auc
#     # print(f"Dataset {dataset_names[i]}: Test AUC: {roc_auc_score(y_trains[i], y_predict_proba)}")



In [136]:
##給xgboost用
y_predicts=[]
for i in range(len(dataset_names)):
    # print(X_tests[i])
    y_predict_proba=models[i].predict_proba(X_tests[i])[:, 1]
    df = pd.DataFrame(y_predict_proba, columns=['y_predict_proba'])
    y_predicts.append(df)
    

## Save result

In [137]:
for idx,dataset_name in enumerate(dataset_names):
    df=y_predicts[idx]
    df.to_csv(f'../Competition_data/{dataset_name}/y_predict.csv', index=False,header=True)

In [138]:
# from sklearn.metrics import roc_auc_score

# test_aucs = []
# for i in range(len(dataset_names)):
#     # 使用模型進行預測，獲得類別 1 的預測機率
#     y_predict_proba = models[i].predict_proba(X_tests[i])[:, 1]
    
#     # 計算 AUC 分數
#     auc = roc_auc_score(y_tests[i], y_predict_proba)
#     print(f'AUC of dataset {i:2}: \t{auc}')
    
#     test_aucs.append(auc)

# # 平均 AUC
# avg_test_auc = sum(test_aucs) / len(test_aucs)
# print("\nAverage Test AUC:", avg_test_auc)
